# R(2+1)D Model Fine-tuning on HMDB51  

In this notebook, we show how to finetune the pretrained R(2+1)D model. We use [HMDB51](http://serre-lab.clps.brown.edu/resource/hmdb-a-large-human-motion-database/) human action dataset for this example.

In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import sys
sys.path.append("../../")
import time
import os
import numpy as np
from sklearn.metrics import accuracy_score
import torch
import torch.cuda as cuda
import torch.nn as nn
import torchvision

from utils_cv.action_recognition.dataset import show_batch, VideoDataset
from utils_cv.action_recognition.model import R2Plus1D 
from utils_cv.common.gpu import system_info

system_info()

### Prerequisites
* Download and extract HMDB51 videos under `./data/hmdb51/videos` directory. 

In [ ]:
from utils_cv.common.data import data_path

In [ ]:
DATA_ROOT = os.path.join(str(data_path()), "misc/hmdb51")
VIDEO_DIR = os.path.join(DATA_ROOT, "videos")
# This split is known as "split1"
TRAIN_SPLIT = os.path.join(DATA_ROOT, "hmdb51_vid_train_split_1.txt")
TEST_SPLIT = os.path.join(DATA_ROOT, "hmdb51_vid_val_split_1.txt")

## Model and Training Configurations

Using the paramters below, one can reproduce published results on the HMDB-51 dataset. Video-clip accuracy of close to 80%  can be obtained using as input to the DNN 32 consecutive frames (`MODEL_INPUT_SIZE`) and running 48 training epochs. Even using only 8 epochs the accuracy already reaches 75%. Inference and training speed can be reduced by a factor of 3-4 by setting `MODEL_INPUT_SIZE=8` and using higher batch size, however at a significant loss in accuracy.

In [ ]:
# Number of consecutive frames used as input to the DNN. Recommended: 32 for high accuracy, 8 for inference speed.
MODEL_INPUT_SIZE = 8

# Batch size. Reduce if running out of memory.
BATCH_SIZE = 8        

# Number of training epochs
EPOCHS = 8

# Model configuration
r2plus1d_hmdb51_cfgs = dict(
    # HMDB51 dataset spec
    num_classes=51,
    video_dir=VIDEO_DIR,
    train_split=TRAIN_SPLIT,
    valid_split=TEST_SPLIT,
    # Pre-trained model spec ("Closer look" and "Large-scale" papers)
    base_model='ig65m',
    sample_length=MODEL_INPUT_SIZE,     
    sample_step=1,        # Frame sampling step
    im_scale=128,         # After scaling, the frames will be cropped to (112 x 112)
    mean=(0.43216, 0.394666, 0.37645),
    std=(0.22803, 0.22145, 0.216989),
    random_shift=True,
    temporal_jitter_step=2,    # Temporal jitter step in frames (only for training set)
    flip_ratio=0.5,
    random_crop=True,
    video_ext='avi',
)

# Training configuration
train_cfgs = dict(
    epochs=EPOCHS,                       # Number of training epochs
    lr=0.0001,                           # Learning rate
    lr_step_size = np.ceil(2/3*EPOCHS),  # Reduce learning rate by factor of 10 after this many epochs.
    batch_size=BATCH_SIZE,               # Model batch size. Reduce if model cannot fit into memory.
    
    # No need to change these parameters
    grad_steps=2,          # Accumulate the gradients over multiple forward passes
    mixed_prec=False,      # Use mixed precision to speed up training
)

## Load Model and Data

Load R(2+1)D 34-layer model pre-trained on IG65M. There are two versions of the model: 8-frame model and 32-frame model based on the input clip length.

In [ ]:
learn = R2Plus1D(r2plus1d_hmdb51_cfgs)

Model structure

In [ ]:
learn.model

Three examples of training (transformed) clips. 

In [ ]:
learn.show_batch(num_samples=3)

## Fine-tuning

In [ ]:
learn.fit(train_cfgs)

Each accuracy is averaged batch-wise accuracy.

## Model Test

Reported accuracy from "Closer look" paper: 74.5% (clip accuracy of 66.1% on split1 based on VMZ repo)

1. sample 10 clips uniformly sampled from each test video: [10 x 3 x (8 or 32) x 112 x 112]
2. calculate clip-level accuracy: Use 10 batch and infer
3. calculate video-level accuracy by averaging them
4. average over the clips

In [ ]:
# ### Load Saved Weights if needed
#learn.load(body_train_cfgs['model_name'] + "_032", body_train_cfgs['model_dir'])

In [ ]:
if cuda.is_available():
    device = torch.device("cuda")
    num_gpus = cuda.device_count()
    # Look for the optimal set of algorithms to use in cudnn. Use this only with fixed-size inputs.
    torch.backends.cudnn.benchmark = True
else:
    device = torch.device("cpu")
    num_gpus = 0

In [ ]:
# Uniformly sample 10 clips per videos ("Closer look" papers)
num_segments = 10
test_ds = VideoDataset(
    split_file=r2plus1d_hmdb51_cfgs['valid_split'],
    video_dir=r2plus1d_hmdb51_cfgs['video_dir'],
    num_segments=num_segments,
    sample_length=r2plus1d_hmdb51_cfgs['sample_length'],
    sample_step=1,
    input_size=112,
    im_scale=r2plus1d_hmdb51_cfgs['im_scale'],
    resize_keep_ratio=True,
    mean=r2plus1d_hmdb51_cfgs['mean'],
    std=r2plus1d_hmdb51_cfgs['std'],
    random_shift=False,
    temporal_jitter=False,
    flip_ratio=0.0,
    random_crop=False,
    random_crop_scales=None,
    video_ext=r2plus1d_hmdb51_cfgs['video_ext'],
)

print("{} samples of {}".format(len(test_ds), test_ds[0][0][0].shape))

show_batch(
    test_ds[0][0],
    r2plus1d_hmdb51_cfgs['sample_length'],
    r2plus1d_hmdb51_cfgs['mean'],
    r2plus1d_hmdb51_cfgs['std']
)

In [ ]:
# Initialize the model
model = learn.model
model.to(device)
if num_gpus > 1:
    model = nn.DataParallel(model)
model.eval()

# Loop over all examples in the test set and compute accuracies
infer_times = []
video_preds = []
video_trues = []
clip_preds = []
clip_trues = []
report_every = 100

with torch.no_grad():
    for i in range(1, len(test_ds)): #[::10]:  # Skip some examples to speed up accuracy computation
        if i % report_every == 0:
            print("Processsing {} of {} samples..".format(i, len(test_ds)))
        
        # Get model inputs
        inputs, label = test_ds[i]
        inputs = inputs.to(device, non_blocking=True)
        
        # Run inference
        start_time = time.time()
        outputs = model(inputs)
        outputs = outputs.cpu().numpy()
        infer_time = time.time() - start_time
        infer_times.append(infer_time)
        
        # Store results
        video_preds.append(outputs.sum(axis=0).argmax())
        video_trues.append(label)
        clip_preds.extend(outputs.argmax(axis=1))
        clip_trues.extend([label] * num_segments)
        
print("Avg. inference time per video (10 clips) =", np.array(infer_times).mean() * 1000, "ms")
print("Video prediction accuracy =", accuracy_score(video_trues, video_preds))
print("Clip prediction accuracy =", accuracy_score(clip_trues, clip_preds))